# Launching Webservers
Webservers are useful platforms for deploying models.  Here we'll show some simple config for a webserver called "flask"

In [6]:
! cat hello_world.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'HELLO WORLD'

if __name__ == "__main__":
    app.run()


Flask apps use the `@app.route` decorator to designate a handler for a path.  In this case, we're defining one for the root, or `/` path.

In [8]:
! export FLASK_APP=hello_world.py && flask run

 * Serving Flask app "hello_world.py"
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Mar/2021 20:30:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2021 20:30:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2021 20:30:38] "GET /foo HTTP/1.1" 404 -
^C


We can see our serialized pipeline.  There's two good options for serializing sklearn pipelines... "joblib" and "pickle".  As you can see, there's not much difference in size for this example.

In [9]:
! ls -lsh | grep train_pipe

   8 -rw-r--r--  1 jdonaldson  staff   1.9K Mar  3 15:51 train_pipe.joblib
   8 -rw-r--r--  1 jdonaldson  staff   1.8K Mar  3 15:35 train_pipe.pkl


Here's a slightly more complex one that permits a `POST` method to send larger amounts of data to the webserver.  We'll use this method to:
1. Send the input data payload
2. Deserialize the data payload into the raw data
3. Use the deserialized pipeline to make a new prediction from the raw data
4. Serialize the prediction
5. Send the serialized prediction back to the reqeuster

In [10]:
! cat app.py

from flask import Flask, request
import pandas as pd
import joblib
import json
import pipeline
from pipeline import *

app = Flask(__name__)
pipe = joblib.load("train_pipe.joblib")

@app.route('/', methods = ['GET', 'POST'])
def index():
    if request.method == "POST":
        dat = pd.read_json(request.form["input"])
        return json.dumps(pipe.predict(dat).tolist())
    else:
        print("Remember to use the POST method for this server")

if __name__ == "__main__":
    app.run()


In [11]:
! python app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Mar/2021 20:43:25] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [03/Mar/2021 20:45:10] "POST / HTTP/1.1" 200 -
^C
